# Multivariate Normal likelihood examples

Assume $$X_1, \ldots, X_n \overset{\text{iid}}{\sim} \text{Normal}(\mu, \Lambda^{-1})$$

$\mu$ is a mean $d \times 1$ *vector* and $\Lambda$ is a $d \times d$ precision *matrix*. 



The normal likelihood for one data *vector* (probably a row in a data frame) is
$$
f(x_i \mid \mu, \Lambda) = (2 \pi)^{-d/2} \det(\Lambda)^{1/2} \exp\left[ - \frac{1}{2}(x_i - \mu)^\intercal \Lambda(x_i - \mu) \right]
$$

The likelihood $p(x_1, \ldots, x_n \mid \mu, \Lambda)$ is 

$$
f(x_1, \ldots, x_n \mid \mu, \Lambda) = (2 \pi)^{-nd/2} \det(\Lambda)^{n/2} \exp\left[ - \frac{1}{2}\sum_{i=1}^n (x_i - \mu)^\intercal \Lambda(x_i - \mu) \right]
$$





NB: *evaluating* the above function can get so close to $0$, that the computer will incorrectly round down to $0$.  This is numerical underflow. It's bad, so to avoid it we'll work with the natural log of the above. 

This isn't really an issue when you're dealing with conjugate priors, bt it will be when we aren't later on. We don't require *evaluation* of the normal density yet.


In [1]:
import numpy as np
import pandas as pd

# pretend this data comes from real life and we don't know the parameters
n = 1000
d = 2
x_data = np.random.normal(loc=0.0, scale=2.0, size=(n,d))
x_data = pd.DataFrame(x_data)
x_data.head()

,0,1
0,2.363376,0.550225
1,-0.133325,-1.527546
2,-1.280575,2.547358
3,-1.777862,-0.998420
4,0.493537,-2.375177


### case 1: known precision matrix

let's say we know the precision of the data: $\Lambda = \begin{bmatrix}1 & 0 \\ 0 & 1 \end{bmatrix}$

then we only have to put a prior on $\mu$. The conjugate prior is normal.

$$
p(\mu) = \text{Normal}(\mu_0, \Lambda_0)
$$

we pick $\mu_0$ and $\Lambda_0$.

#### Bayes rule:

$$
p(\mu \mid x_1, \ldots, x_n) \propto p(x_1, \ldots, x_n \mid \mu)p(\mu) 
$$

which gives us this specific normal posterior:

$$
\mu \mid x_1, \ldots, x_n \sim \text{Normal}\left(\mu_0\left[\frac{\Lambda_0}{n\Lambda + \Lambda_0} \right] + \bar{x} \left[\frac{n\Lambda}{n\Lambda + \Lambda_0} \right],
\left[ n\Lambda + \Lambda_0\right]^{-1}
\right)
$$

This is very similar to the univariate case 1 in the other document. We just replace univariate precisions with matrix precisions. Computationally, remember to use matrix multiplication (`@`) instead of elementwise multiplication (`*`).


In [2]:
# when we use conjugate priors, we only need to use Python like a simple calculator
xbar = x_data.mean().values
Lambda0 = np.array([[1,0],[0,1]]) # chosen hyperparameter
mu0 = np.array([0,0])  # chosen hyperparameter
Lambda = np.array([[1,0],[0,1]]) # assumed known
# "getting" the posterior is simply invoking the formula
posterior_mean = xbar@(n*Lambda) @ np.linalg.inv(Lambda0 + n*Lambda ) + mu0@Lambda0@ np.linalg.inv(Lambda0 + n*Lambda )
posterior_precision = Lambda0 + n*Lambda 
posterior_covariance_matrix = np.linalg.inv(posterior_precision)
print("bayesian mean estimate: ", posterior_mean)
print("frequentist mean estimate", xbar)

bayesian mean estimate:  [-0.00803584  0.09730708]
frequentist mean estimate [-0.00804388  0.09740438]


### case 2: unknown precision/variance

let's say we don't know the precision matrix of the data $\Lambda$. Then we have to have a prior for both $\mu$ and $\Lambda$. Everything that's unknown goes into the prior.

The conjugate prior is 

$$
p(\mu \mid \Lambda)p(\Lambda) = \text{Normal}(\mu_0, \frac{1}{\kappa_0}\Lambda^{-1})\text{Wishart}(\Lambda_0, \nu_0)
$$

Notice that

- a "Wishart" is kind of like a multivariate "Gamma". Gamma is for positive random variables, Wishart is for "positive-definite" random matrices.
- $\mu$ and $\Lambda$ are the unknown parameters
- $\mu_0, \kappa_0, \Lambda_0$ and $\nu_0$ are chosen by us
- I like to simulate $\mu$ and $\Lambda$ after I choose prior hyperparameters just to make sure it looks like the values look sensible
- Additionally, $E[\Lambda] = \Lambda_0 \nu_0$


In [3]:
"""
Picking the prior
"""

from scipy.stats import wishart
from scipy.stats import multivariate_normal

# chosen prior hyperparameters
mu_0 = np.array([2,3])
kappa_0 = 3
Lambda_0 = np.array([[1,0],[0,1]])
nu_0 = 2

# simulate parameters from the prior
def sim_prior(mu_0, kappa_0, Lambda_0, nu_0):
    # sim Lambda
    Lambda = wishart.rvs(df = nu_0, scale = Lambda_0, size = 1)
    # sim mu given Lambda
    mu = multivariate_normal.rvs(mean=mu_0, cov=np.linalg.inv(Lambda), size=1)
    return mu, Lambda

# do these look like reasonable values of mu and Lambda?
sim_prior(mu_0, kappa_0, Lambda_0, nu_0)

(array([3.04366694, 2.51588385]),
 array([[2.18887717, 1.4885916 ],
        [1.4885916 , 2.18117816]]))

#### Getting the posterior

We'll end up with a product posterior. Product one will be a multivariate normal, and product two will be a Wishart. It's a conjugate prior, so the prior and posterior are the same distribution.

$$
p(\mu, \Lambda \mid x_1, \ldots, x_n) = p(\mu \mid \Lambda, x_1, \ldots, x_n)p( \Lambda \mid x_1, \ldots, x_n)
$$

#### things to notice:

The overall/joint posterior $p(\mu, \Lambda \mid x_1, \ldots, x_n) $ is not normal. It doesn't make sense to have a normal distribution for a thing that must be "positive" $\Lambda$.

The *conditional* posterior $p(\mu \mid \Lambda, x_1, \ldots, x_n)$ is normal, but it depends on a thing you don't really know $\Lambda$ 

The *conditional* posterior $p(\mu \mid \Lambda, x_1, \ldots, x_n)$  has the same form as the above situation with known variance/precision.

The *marginal* posterior $p(\Lambda \mid x_1, \ldots, x_n)$ is Wishart. 

The four *prior* hyperparameters are $\mu_0, \kappa_0, \Lambda_0$ and $\nu_0$

These turned into four *posterior* hyperparameters $\frac{n}{n + \kappa_0} \bar{x} + \frac{ \kappa_0}{n + \kappa_0} \mu_0$, $n + \kappa_0$, $\left( 
\Lambda_0^{-1} +
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
+
\frac{n\kappa_0}{n + \kappa_0} 
\left(\bar{x} -  \mu_0\right)
\left(\bar{x} -  \mu_0\right)^\intercal 
\right)^{-1}$ and $n + \nu_0$

Sometimes they call the posterior hyper parameters with  $\mu_n, \kappa_n, \Lambda_n$ and $\nu_n$. This is shorter, but it doesn't tell you the update formulae.



In [7]:
# we picked prior hyperparameters in the previous cell
# calculate posterior hyperparameters
mu_n = xbar * (n/(kappa_0 + n )) + mu_0*(kappa_0/(kappa_0 + n ))
kappa_n = n + kappa_0
nu_n = n + nu_0
Lambda_n_inv = Lambda_0 + (x_data - xbar).T @ (x_data - xbar) + (n*kappa_0)/(n+kappa_0)*np.outer(xbar - mu_0, xbar - mu_0)
Lambda_n = np.linalg.inv(Lambda_n_inv)

# if you pick noninformative priors, they should be roughly the same
print("Bayesian posterior mean: ", mu_n)
print("Frequentist mean estimate: ", xbar)


Bayesian posterior mean:  [-0.00203777  0.10608613]
Frequentist mean estimate:  [-0.00804388  0.09740438]


In [5]:
# proofs below are commented out...you can uncomment them to see them if you're interested

<!-- ### Proof of posterior for unknown precision matrix case (case 2)



First find part 1/2 conditional posterior $p(\mu \mid \Lambda, x_1, \ldots, x_n)$. We can ignore $\Lambda$ pieces here.

\begin{align*}
p(\mu \mid \Lambda, x_1, \ldots, x_n)
&\propto 
f(x_1, \ldots, x_n \mid \mu, \Lambda) p(\mu \mid \Lambda)p(\Lambda) \\
&\propto 
f(x_1, \ldots, x_n \mid \mu, \Lambda^{-1}) p(\mu \mid \Lambda) \\
&\propto
\exp\left(-\frac{1}{2}\sum_i (x_i - \mu)^\intercal\Lambda(x_i- \mu) \right)
\exp\left[-\frac{\kappa_0}{2} (\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0)\right]\\
&=
\exp\left(-\frac{1}{2}
\left\{
\sum_i (\mu - x_i)^\intercal\Lambda(\mu - x_i)
+
\kappa_0(\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0)
\right\}
 \right)
\\
&\propto
\exp\left(-\frac{1}{2}
\left\{
n \mu^\intercal \Lambda \mu 
-2n \bar{x}^\intercal\Lambda \mu 
+
\kappa_0(\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0)
\right\}
 \right)
\\
&\propto
\exp\left(-\frac{1}{2}
\left\{
n \mu^\intercal \Lambda \mu 
-2n \bar{x}^\intercal\Lambda \mu 
+
\kappa_0 \mu^\intercal  \Lambda \mu
- 2 \kappa_0 \mu_0^\intercal \Lambda \mu
\right\}
 \right)
\\
&=
\exp\left(-\frac{1}{2}
\left\{
\mu^\intercal\left[ (n + \kappa_0) \Lambda \right] \mu 
-2\left[ n \bar{x} + \kappa_0 \mu_0 \right]^\intercal\Lambda \mu 
\right\}
 \right)
\\
&=
\exp\left(
-\frac{1}{2} (\mu - \left[ (n + \kappa_0) \Lambda\right]^{-1} \left[ n \bar{x} + \kappa_0 \mu_0 \right])^\intercal  \Lambda\left[ (n + \kappa_0) \Lambda\right] (\mu - \left[ (n + \kappa_0) \Lambda\right]^{-1} \left[ n \bar{x} + \kappa_0 \mu_0 \right]\Lambda)
\right)
\\
\end{align*}

So
$$
\mu \mid \Lambda, x_1, \ldots, x_n \sim \text{Normal}(
\frac{n}{n + \kappa_0} \bar{x} + \frac{ \kappa_0}{n + \kappa_0} \mu_0, 
\left[ (n + \kappa_0) \Lambda\right]^{-1}
)
$$


Second find part 2/2 the marginal posterior $p(\Lambda \mid x_1, \ldots, x_n)$. 

In this part of the proof, we use the *trace operator* (which takes a matrix and returns the sum of its diagonal elements). The trace operator is used to write down Wishart densities, and our goal is to organize stuff into a Wishart density.


Now

\begin{align*}
p(\Lambda \mid x_1, \ldots, x_n) 
&\propto
\int
p( x_1, \ldots, x_n \mid \mu, \Lambda)p(\mu \mid \Lambda) p(\Lambda) d\mu
\\
&\propto
\int
\det(\Lambda)^{n/2} 
\exp\left(-\frac{1}{2}\sum_i (x_i - \mu)^\intercal\Lambda(x_i- \mu) \right) \times \\
&\hspace{10mm} 
\det(\Lambda)^{1/2}
\exp\left[-\frac{\kappa_0}{2} (\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0)\right] \times \\
&\hspace{10mm} 
\det(\Lambda)^{(\nu_0 - d - 1)/2}
\exp\left[-\frac{1}{2} \text{tr} \left( \Lambda_0^{-1}\Lambda \right) \right]
d\mu \\
&=
\det(\Lambda)^{(n + \nu_0 - d)/2} 
\exp\left[-\frac{1}{2} \text{tr} \left( \Lambda_0^{-1}\Lambda \right) \right]\times \\
&\hspace{10mm} 
\int \exp\left(-\frac{1}{2}\sum_i (x_i - \mu)^\intercal\Lambda(x_i- \mu) \right) 
\exp\left[-\frac{\kappa_0}{2} (\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0)\right] d\mu
\end{align*}

We can try to "recognize" a normal distirbution in that last integral. That will allow us to use the formula for a Gaussian density instead of doing the wrote integration. To clean up the notation, let's look at negative twice the logarithm of the exponential part and play with that. Remember, don't drop anything that involves $\Lambda$.

\begin{align*}
&\sum_i (x_i - \mu)^\intercal\Lambda(x_i- \mu)
+
\kappa_0 (\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0) \\
&=
\sum_i x_i^\intercal\Lambda x_i
- 2 n \bar{x}^\intercal \Lambda \mu
+ n\mu^\intercal \Lambda \mu
+ \kappa_0 \mu^\intercal \Lambda \mu
- 2 \kappa_0 \mu_0^\intercal  \Lambda \mu 
+ \kappa_0 \mu_0^\intercal \Lambda \mu_0 \\
&=
(n + \kappa_0) \mu^\intercal \Lambda \mu
- 2 \left[ n \bar{x} +  \kappa_0 \mu_0 \right]^\intercal\Lambda \mu
+ \underbrace{\sum_i x_i^\intercal\Lambda x_i
+
\kappa_0 \mu_0^\intercal \Lambda \mu_0 }_{\text{no $\mu$}} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\sum_i x_i^\intercal\Lambda x_i
+
\kappa_0 \mu_0^\intercal \Lambda \mu_0 
-
\frac{1}{n + \kappa_0}
\{ n \bar{x} +  \kappa_0 \mu_0 \}^\intercal
\Lambda 
\{ n \bar{x} +  \kappa_0 \mu_0 \} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\sum_i x_i^\intercal\Lambda x_i
+
\kappa_0 \mu_0^\intercal \Lambda \mu_0 
-
\frac{1}{n + \kappa_0}
\left\{
n^2 \bar{x}^\intercal \Lambda \bar{x}
+ 2 n \bar{x}^\intercal \Lambda \kappa_0 \mu_0
+  \kappa_0^2 \mu_0^\intercal \Lambda \mu_0
\right\} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\text{tr} \left\{\left[
\sum_i x_i x_i^\intercal
+
\kappa_0 \mu_0 \mu_0^\intercal 
-
\frac{n^2 }{n + \kappa_0} \bar{x} \bar{x}^\intercal 
- 2\frac{n \kappa_0}{n + \kappa_0}   \mu_0 \bar{x}^\intercal
-  \frac{\kappa_0^2}{n + \kappa_0}  \mu_0 \mu_0^\intercal 
\right]
\Lambda \right\} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\text{tr} \left\{\left[
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
+
\kappa_0 \mu_0 \mu_0^\intercal 
-
\frac{n^2 - n(n + \kappa_0)}{n + \kappa_0} \bar{x} \bar{x}^\intercal 
- 2\frac{n \kappa_0}{n + \kappa_0}   \mu_0 \bar{x}^\intercal
-  \frac{\kappa_0^2}{n + \kappa_0}  \mu_0 \mu_0^\intercal 
\right]
\Lambda \right\} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\text{tr} \left\{\left[
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
-
\frac{n^2 - n(n + \kappa_0)}{n + \kappa_0} \bar{x} \bar{x}^\intercal 
- 2\frac{n \kappa_0}{n + \kappa_0}   \mu_0 \bar{x}^\intercal
-  \frac{\kappa_0^2 - \kappa_0(n + \kappa_0)}{n + \kappa_0}  \mu_0 \mu_0^\intercal 
\right]
\Lambda \right\} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\text{tr} \left\{\left[
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
+
\frac{n\kappa_0}{n + \kappa_0} \bar{x} \bar{x}^\intercal 
- 2\frac{n \kappa_0}{n + \kappa_0}   \mu_0 \bar{x}^\intercal
+   \frac{n \kappa_0}{n + \kappa_0}  \mu_0 \mu_0^\intercal 
\right]
\Lambda \right\} \\
&=
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]^\intercal
\left[ (n + \kappa_0)\Lambda \right]
\left[\mu - \{ \frac{n}{n + \kappa_0} \bar{x} +  \frac{\kappa_0}{n + \kappa_0} \mu_0 \} \right]
+\text{tr} \left\{\left[
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
+
\frac{n\kappa_0}{n + \kappa_0} 
\left(\bar{x} -  \mu_0\right)
\left(\bar{x} -  \mu_0\right)^\intercal \right]
\Lambda \right\} \\
\end{align*}


So we can finish it off now

\begin{align*}
&\det(\Lambda)^{(n + \nu_0 - d)/2} 
\exp\left[-\frac{1}{2} \text{tr} \left( \Lambda_0^{-1}\Lambda \right) \right]\times \\
&\hspace{10mm} 
\int \exp\left(-\frac{1}{2}\sum_i (x_i - \mu)^\intercal\Lambda(x_i- \mu) \right) 
\exp\left[-\frac{\kappa_0}{2} (\mu - \mu_0)^\intercal  \Lambda (\mu - \mu_0)\right] d\mu \\
&\propto
\det(\Lambda)^{(n + \nu_0 - d - 1)/2} 
\exp\left[
-\frac{1}{2} \text{tr}
\left\{
\Lambda_0^{-1}
+
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
+
\frac{n\kappa_0}{n + \kappa_0} 
\left(\bar{x} -  \mu_0\right)
\left(\bar{x} -  \mu_0\right)^\intercal 
\right\}\Lambda 
\right]
\end{align*}

This means

$$
\Lambda \mid x_1, \ldots, x_n \sim \text{Wishart}\left(n + \nu_0, \left( 
\Lambda_0^{-1} +
\sum_i (x_i - \bar{x}) (x_i - \bar{x})^\intercal
+
\frac{n\kappa_0}{n + \kappa_0} 
\left(\bar{x} -  \mu_0\right)
\left(\bar{x} -  \mu_0\right)^\intercal 
\right)^{-1}
\right)
$$

 -->

In [6]:
# proofs in cell above ^